# Requirements

In [ ]:
import matplotlib.pyplot as plt
import random
%matplotlib inline

In [ ]:
%load_ext line_profiler

# Pure Python implementation

## Implementation

First we define a class to evolve the cellular autotomaton.  A runner is created for a specific rule, specified by a number between 0 and 255.  For instance, rule 47 would translate to:
* $000 \mapsto 1$
* $001 \mapsto 1$
* $010 \mapsto 1$
* $011 \mapsto 1$
* $100 \mapsto 0$
* $101 \mapsto 1$
* $110 \mapsto 0$
* $111 \mapsto 0$

In [ ]:
class AutomatonRunner:
    
    def __init__(self, rule_nr):
        self._rules = []
        for _ in range(8):
            self._rules.append(rule_nr % 2)
            rule_nr //= 2
    
    def random_automaton(self, nr_cells, seed=None):
        if seed is not None:
            random.seed(seed)
        return random.choices((0, 1), k=nr_cells)
    
    def _next_generation(self, automaton):
        idx = (automaton[-1] << 2) | (automaton[0] << 1) | automaton[1]
        ng_automaton = [self._rules[idx]]
        for i in range(1, len(automaton) - 1):
            idx = (automaton[i - 1] << 2) | (automaton[i] << 1) | automaton[i + 1]
            ng_automaton.append(self._rules[idx])
        idx = (automaton[-2] << 2) | (automaton[-1] << 1) | automaton[0]
        ng_automaton.append(self._rules[idx])
        return ng_automaton
                                
    def evolve(self, automaton, nr_generations, handler):
        if not handler(automaton): return
        for _ in range(nr_generations):
            automaton = self._next_generation(automaton)
            if not handler(automaton): return
    
    def evolve_random(self, nr_cells, nr_generations, handler, seed=None):
        automaton = self.random_automaton(nr_cells, seed)
        self.evolve(automaton, nr_generations, handler)
        
    def __str__(self):
        auto_str = ''
        for i, result in enumerate(self._rules):
            auto_str += f'{i//4 % 2}{i//2 % 2}{i % 2} -> {result}\n'
        return auto_str

To verify the implementation, we check whether the rules have been created correctly.

In [ ]:
runner = AutomatonRunner(47)

In [ ]:
print(str(runner))

## Handlers

We define two handlers, one for benchmarking purposes that does nothing, the other gathers the successive generations into a list.

In [ ]:
def do_nothing_handler(automaton):
    return True

In [ ]:
class GenerationsHandler:
    
    def __init__(self):
        self._generations = []
        
    def __call__(self, automaton):
        self._generations.append(automaton)
        return True
        
    @property
    def generations(self):
        return self._generations
    
    def visualize(self):
        plt.imshow(self._generations);

## Running an automaton

In [ ]:
runner = AutomatonRunner(47)

We create an automaton that is reandomly initialized.

In [ ]:
automaton = runner.random_automaton(10)

In [ ]:
generations = GenerationsHandler()
runner.evolve(automaton, 10, generations)

In [ ]:
generations.visualize()

## Exploring rules

In [ ]:
nr_cols = 16
nr_generations = 80
figure, axes = plt.subplots(256//nr_cols, nr_cols, figsize=(100, 100))
automaton = random.choices((0, 1), k=20)
for rule_nr in range(256):
    row_nr = rule_nr//nr_cols
    col_nr = rule_nr % nr_cols
    runner = AutomatonRunner(rule_nr)
    generations = GenerationsHandler()
    runner.evolve(automaton, nr_generations, generations)
    axes[row_nr][col_nr].imshow(generations.generations, aspect='auto')
    axes[row_nr][col_nr].set_title(str(rule_nr));
plt.tight_layout()

Rule 129 seems to be interesting.

In [ ]:
runner = AutomatonRunner(129)

In [ ]:
generations = GenerationsHandler()
runner.evolve_random(nr_cells=200, nr_generations=300, handler=generations, seed=1234)
plt.imshow(generations.generations);

## Performance

We can measure the performance by running a large automaton for a considerable number of generations.

In [ ]:
runner = AutomatonRunner(129)

In [ ]:
nr_cells, nr_generations = 10_000, 500

In [ ]:
%timeit _ = runner.evolve_random(nr_cells=nr_cells, nr_generations=nr_generations, handler=do_nothing_handler, seed=1234)

In [ ]:
%prun _ = runner.evolve_random(nr_cells, 500, do_nothing_handler)

In [ ]:
%lprun -f runner._next_generation runner.evolve_random(nr_cells, 50, do_nothing_handler)

# Using numpy

## Implementation

Using lists to represent an automaton is probably not the best idea, so we can replace them by numpy arrays.

In [ ]:
import numpy as np

In [ ]:
class AutomatonRunnerNumpy(AutomatonRunner):
    
    def __init__(self, rule_nr):
        self._rules = np.empty(8, dtype=np.int32)
        for i in range(self._rules.size):
            self._rules[i] = rule_nr % 2
            rule_nr //= 2
    
    def random_automaton(self, nr_cells, seed=None):
        if seed is not None:
            np.random.seed(seed)
        return np.random.randint(0, 2, size=(nr_cells, ))

    def _next_generation(self, automaton):
        ng_automaton = np.empty_like(automaton)
        idx = (automaton[-1] << 2) | (automaton[0] << 1) | automaton[1]
        ng_automaton[0] = self._rules[idx]
        for i in range(1, automaton.size - 1):
            idx = (automaton[i - 1] << 2) | (automaton[i] << 1) | automaton[i + 1]
            ng_automaton[i] = self._rules[idx]
        idx = (automaton[-2] << 2) | (automaton[-1] << 1) | automaton[0]
        ng_automaton[-1] = self._rules[idx]
        return ng_automaton

In [ ]:
runner = AutomatonRunnerNumpy(129)

In [ ]:
generations = GenerationsHandler()
runner.evolve_random(nr_cells=200, nr_generations=300, handler=generations, seed=1234)

In [ ]:
generations.visualize()

## Performance

We can measure the performance by running a large automaton for a considerable number of generations.

In [ ]:
runner = AutomatonRunnerNumpy(129)

In [ ]:
nr_cells, nr_generations = 10_000, 500

In [ ]:
%timeit _ = runner.evolve_random(nr_cells, nr_generations, do_nothing_handler, seed=1234)

In [ ]:
%prun runner.evolve_random(nr_cells, 50, do_nothing_handler)

In [ ]:
%lprun -f runner._next_generation runner.evolve_random(nr_cells, 50, do_nothing_handler)

# Using numba

## Implementation

In [ ]:
from numba import njit

In [ ]:
@njit
def next_generation(rules, automaton):
    idx = (automaton[-1] << 2) | (automaton[0] << 1) | automaton[1]
    ng_automaton = [rules[idx]]
    for i in range(1, len(automaton) - 1):
        idx = (automaton[i - 1] << 2) | (automaton[i] << 1) | automaton[i + 1]
        ng_automaton.append(rules[idx])
    idx = (automaton[-2] << 2) | (automaton[-1] << 1) | automaton[0]
    ng_automaton.append(rules[idx])
    return ng_automaton

In [ ]:
class AutomatonRunnerNumba(AutomatonRunner):
    
    def _next_generation(self, automaton):
        return next_generation(self._rules, automaton)

In [ ]:
runner = AutomatonRunnerNumba(129)

In [ ]:
generations = GenerationsHandler()
runner.evolve_random(200, 300, generations, seed=1234)
generations.visualize()

## Performance

In [ ]:
runner = AutomatonRunnerNumba(129)

In [ ]:
nr_cells, nr_generations = 10_000, 500

In [ ]:
%timeit _ = runner.evolve_random(nr_cells, nr_generations, do_nothing, seed=1234)